In [2]:
import owlready2 as owl
from owlready2 import *

import types
from types import NoneType

import numpy as np
import matplotlib.pyplot as plt
import os
from datetime import datetime
import random

* Owlready2 * Warning: optimized Cython parser module 'owlready2_optimized' is not available, defaulting to slower Python implementation




In [3]:
# dir = '/Users/victorlacerda/Documents/VSCode/ELH Implementation/NormalizedOntologies/goslimyeast.xml.owl'
# dir = '/Users/victorlacerda/Documents/VSCode/ELH Implementation/NormalizedOntologies/galennorm.xml.owl'
# dir = '/Users/victorlacerda/Documents/VSCode/ELH Implementation/NormalizedOntologies/gonorm.xml.owl'
# dir = '/Users/victorlacerda/Documents/VSCode/ELH Implementation/NormalizedOntologies/'
# dir = '/Users/victorlacerda/Documents/VSCode/ELH Implementation/NormalizedOntologies/family_ontology.owl'
# dir = '/Users/victorlacerda/Desktop/dbpedia15k.owl'


In [4]:
'''
Functions for checking whether
a given General Class Axiom is 
in normal form. GCAs only have
constructs on the left side of
the inclusion.
'''

# Normal Form 0: A subsumed by B

def is_nf0(tbox_axiom):

    axiom_left_side = tbox_axiom.left_side
    axiom_right_side = tbox_axiom.is_a

    if type(axiom_left_side) != ThingClass:
        return False
    else:
        check = (len(axiom_right_side) == 1 and
                 type(axiom_right_side[0]) == ThingClass
                )
    return check

# Normal Form 1: A1 and A2 subsumed by B with A1, A2 and B in N_C.

def is_nf1(tbox_axiom):
    
    axiom_left_side = tbox_axiom.left_side
    axiom_right_side = tbox_axiom.is_a

    if type(axiom_left_side) != And:
        return False
    else:
        check = (len(axiom_left_side.Classes) == 2 and
                type(axiom_left_side) == And and
                all(type(cls) == ThingClass for cls in axiom_left_side.Classes) and
                len(axiom_right_side) == 1 and
                type(axiom_right_side[0]) == ThingClass
                )
    
    return check

# Normal Form 2: \exists r.A subsumed by B, where A and B \in N_C and r \in N_R.

def is_nf2(tbox_axiom):
    
    axiom_left_side = tbox_axiom.left_side
    axiom_right_side = tbox_axiom.is_a

    if type(axiom_left_side) != Restriction:
        return False
    else:
        check = (axiom_left_side.type == SOME and
                type(axiom_left_side.value) == ThingClass and
                len(axiom_right_side) == 1 and
                type(axiom_right_side[0]) == ThingClass
                )
    
    return check

# Normal Form 3: A subsumed by \exists r.B, where A and B \in N_C and r \in N_R.

def is_nf3(tbox_axiom):

    axiom_left_side = tbox_axiom.left_side
    axiom_right_side = tbox_axiom.is_a

    if type(axiom_left_side) != ThingClass:
        return False
    else:

        check = (len(axiom_left_side.Classes) == 1 and
                type(axiom_left_side.Classes[0]) == ThingClass and
                len(axiom_right_side.Classes) == 1 and
                type(axiom_right_side[0]) == Restriction and
                axiom_right_side[0].type == SOME and
                type(axiom_right_side[0].value) == ThingClass
                )
    
    return check

In [5]:
def check_gcas_normalized(ontology):

    gcas_iter = ontology.general_class_axioms()
    counter = 0
    
    for tbox_axiom in gcas_iter:
        if is_nf0(tbox_axiom) or is_nf1(tbox_axiom) or is_nf2(tbox_axiom) or is_nf3(tbox_axiom):
            counter += 1
        else:
            print('Error')
            break
    
    print('')
    print('The ontologys General Class Axioms are in normal form.')
    print(f'All {counter} General Class Axioms were checked.')
    return True


In [6]:
'''
Functions for checking whether
a concept inclusion is in normal
form. Concept inclusions only
have concept names (classes of
type ThingClass) on their left
side.
'''

def check_subclassof_normalized(ontology):

    classes = ontology.classes()
    counter = 0

    for concept_name in classes:
        axiom_right_side = list(concept_name.is_a) # We do not need to check the left side as it is guaranteed to be a concept name.

        for concept in axiom_right_side:
            
            if type(concept) == And:
                print('The ontology is not normalized.')
                print('Found axiom of the type A \sqsubseteq C \sqcap D.')
                break
            
            elif type(concept) == Restriction and type(concept.value) != ThingClass:
                print('The ontology is not normalized')
                print('Found axiom of the type A \sqsubseteq \exists r.C, where C is complex.')
                break

            elif type(concept) == Restriction and type(concept.value) == ThingClass:
                counter += 1
                pass

            elif type(concept) == ThingClass:
                counter += 1
                pass
    
    print('')
    print(f'The ontologys concept inclusions are normalized.')
    print(f'All {counter} concept inclusions with concept names on the left side have been checked.')
    return True            

In [7]:
def check_ontology_normalized(onto_dir):

    onto = get_ontology(dir)
    onto = onto.load()

    if check_gcas_normalized(onto) and check_subclassof_normalized(onto):
        print('')
        print('================================================================================')
        print('The ontology is in normal form.')

In [8]:
check_ontology_normalized(dir)


The ontologys General Class Axioms are in normal form.
All 0 General Class Axioms were checked.

The ontologys concept inclusions are normalized.
All 2507 concept inclusions with concept names on the left side have been checked.

The ontology is in normal form.
